In [1]:
'''
Date: 7/Dec/2022
Author: Yike Xie
Content: prepare cellranger v3 mtx files of each sample
'''

'\nDate: 7/Dec/2022\nAuthor: Yike Xie\nContent: prepare cellranger v3 mtx files of each sample\n'

In [2]:
import os

import numpy as np
import pandas as pd

import anndata
import scanpy as sc
import sc_utils 

In [8]:
path = '/home/yike/phd/dengue/data/mergedata_luca_NKT_202206.h5ad'
adata = sc.read_h5ad(path)
adata_kid = adata[adata.obs['platform'] == '10X']

/home/yike/.conda/envs/scanpy/lib/python3.8/site-packages/anndata/_core/anndata.py:1095: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  df_sub[k].cat.remove_unused_categories(inplace=True)


In [5]:
adata_kid.obs.columns

Index(['Age', 'Condition', 'DENV_minus', 'DENV_plus', 'DENV_reads', 'Gender',
       'ID', 'batch', 'cell_quality', 'cell_subtype', 'cell_subtype_2',
       'cell_subtype_zhiyuan', 'cell_type', 'cell_type_zhiyuan',
       'cell_type_new_2', 'doublets', 'mt_frac', 'n_counts', 'n_genes',
       'platform', 'viral_load_nano', 'viral_load_qpcr', 'Dengue_exposure',
       'cell_subtype_luca', 'cell_subtype_new', 'cell_type_luca',
       'cell_type_new'],
      dtype='object')

In [9]:
# remove additional information
obs = adata_kid.obs[['cell_type_new', 'cell_subtype_new', 'ID', 'Age', 'Condition', 'Gender', 'DENV_reads', 
                     'mt_frac', 'n_counts', 'n_genes', 'cell_quality', 'doublets', 'batch']].copy()
obs.rename(columns={'cell_type_new': 'cell_type', 'cell_subtype_new': 'cell_subtype', 'Age': 'age', 'Condition': 'condition', 'Gender': 'sex'}, inplace=True)
var = pd.DataFrame([], index=adata_kid.var_names)
adata_kid.obs = obs
adata_kid.var = var

# save as loom, h5ad
print('Writing h5ad')
adata_kid.write('/home/yike/phd/dengue/data/GEO/dengue_kids.h5ad')
print('Writing loom file')
adata_kid.write('/home/yike/phd/dengue/data/GEO/dengue_kids.loom')

Writing h5ad
Writing loom file


In [32]:
# prepare cellranger v3 mtx files of each sample

output_dir = '/home/yike/phd/dengue/data/GEO/mtx/'
for ID in adata_kid.obs['ID'].unique():
    tmp = adata_kid[adata_kid.obs['ID'] == ID]
    path = os.path.join(output_dir, ID)
    if os.path.exists(path) is False:
        os.mkdir(path)
    sc_utils.write_mtx(tmp, path)

In [41]:
for fd in os.listdir(output_dir):
    path = os.path.join(output_dir, fd)
    for fdn in os.listdir(path):
        os.replace(os.path.join(path, fdn), os.path.join(output_dir, fd + '_' + fdn))

In [46]:
data = pd.read_csv('/home/yike/phd/dengue/data/GEO/md5sums_fastq.txt', sep=',')
IDs = adata_kid.obs['ID'].unique().tolist()
dt = []
for name in data['filename']:
    if (name[:8] in IDs) == False:
        dt.append(name[:5] + '_01' + name[5:])
    else:
        dt.append(name)
        
data['filename'] = dt
data.to_csv('/home/yike/phd/dengue/data/GEO/sum.txt', sep=',', index=False)